In [ ]:
import json
from collections import OrderedDict, defaultdict, Counter
from dateutil import parser
from datetime import datetime
import pickle as pkl

import pandas as pd
import numpy as np
from scipy.stats import pearsonr, spearmanr, linregress

import matplotlib.pyplot as plt
import matplotlib.dates as mdates
from mpl_toolkits.mplot3d import Axes3D
from matplotlib.ticker import MaxNLocator
from matplotlib.patches import Rectangle
import seaborn as sns
from pandas.plotting import register_matplotlib_converters
import gzip

from tqdm import tqdm
import glob
import os
import subprocess

import socialsim as ss
register_matplotlib_converters()
sns.set()

In [ ]:
# append = 'to_5_31'
# append = 'to_6_29'
# append = 'to_7_06'
# append = 'to_7_13'
# append = 'to_7_20'
# append = 'to_7_27'
append = 'to_8_03'

In [ ]:
# idx = pd.date_range(pd.to_datetime('2020-03-30'), pd.to_datetime('2020-05-31'))
# idx = pd.date_range(pd.to_datetime('2020-03-30'), pd.to_datetime('2020-06-29'))
# idx = pd.date_range(pd.to_datetime('2020-03-30'), pd.to_datetime('2020-07-06'))
# idx = pd.date_range(pd.to_datetime('2020-03-30'), pd.to_datetime('2020-07-13'))
# idx = pd.date_range(pd.to_datetime('2020-03-30'), pd.to_datetime('2020-07-20'))
# idx = pd.date_range(pd.to_datetime('2020-03-30'), pd.to_datetime('2020-07-27'))
idx = pd.date_range(pd.to_datetime('2020-03-30'), pd.to_datetime('2020-08-03'))

In [ ]:
with open('/data/leidos_extracted/2021CP5/cp5_eval_nodes.txt', 'r') as f:
    nodes = f.read().split('\n')[:-2]
with open('/data/leidos_extracted/2021CP5/cp5_other_nodes.txt', 'r') as f:
    nodes += f.read().split('\n')[:-1]
nodes

In [ ]:
with open('/data/leidos_extracted/2021CP5/twitter_time_series_{}.json'.format(append), 'r') as f:
    d = json.loads(f.read())
tdf = {k: pd.read_json(v, orient='columns') for k, v in d.items()}
with open('/data/leidos_extracted/2021CP5/youtube_time_series_{}.json'.format(append), 'r') as f:
    d = json.loads(f.read())
ydf = {k: pd.read_json(v, orient='columns') for k, v in d.items()}

In [ ]:
# with open('/data/leidos_extracted/2021CP5/gdelt_time_series.json', 'r') as f:
#     d = json.loads(f.read())
# gdf = {k: pd.read_json(v, typ='series') for k, v in d.items()}
with open('/data/leidos_extracted/2021CP5/acled_time_series.json', 'r') as f:
    d = json.loads(f.read())
adf = {k: pd.read_json(v, typ='series') for k, v in d.items()}

In [ ]:
ycorr = OrderedDict()
for node in nodes:
    if node in ydf:
        z = np.array(ydf[node].reindex(idx, fill_value=0).EventCount.to_list())
        if np.linalg.norm(z) > 0:
            z = z / np.linalg.norm(z)
    else:
        z = np.array(pd.Series().reindex(idx, fill_value=0).to_list())
    tmp = {}
    for ak in adf.keys():
        x = np.pad(np.array(adf[ak].reindex(idx, fill_value=0).to_list()), 1, mode='constant')
        tmp[ak] = pearsonr(x[0:0+len(z)], z)[0]
    ycorr[node] = tmp

In [ ]:
for k in ycorr.keys():
    for kk in ycorr[k].keys():
        if np.isnan(ycorr[k][kk]):
            ycorr[k][kk] = 0.0

In [ ]:
with open('/data/leidos_extracted/2021CP5/youtube_acled_corr_{}.json'.format(append), 'w') as f:
    json.dump(ycorr, f)

In [ ]:
for node in nodes:
    print(node)
    plt.plot(ydf[node].EventCount)
#     k, v = max(ycorr['benefits/development/energy'].items(), key=lambda x: x[1])
    plt.plot(np.sum(adf[ak] * max(0, ycorr[node][ak]) for ak in adf.keys()))
    plt.show()